# Streamlining a state-by-state report with python and InDesign

## Table of Contents
- [Part I - Prep the Datasets](#prep)
- [Part II - Calculate the Numeric Variables](#numeric)
- [Part III - Add the Text Variables](#text)
- [Part IV - Add Columns for InDesign Graphs](#chartwell)
- [Part V - Run a Sample Text Simulation](#simulation)
- [Part VI - Format the Data for your Comms Coworker](#format)
- [Part VII - Export](#export)


In [265]:
import numpy as np
import pandas as pd
import re

### Image of side 1 of the state advocacy page

In [266]:
#Note, this won't appear when converted to html
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "page1_draft.PNG", width=500)

<a id='prep'></a>
## Prep the Datasets

 **IMPORT**

ADD THE STATE DATA

Note - data for states and national have already been prepared in python from the Department of Housing and Urban Development's Annual Homeless Assessment Report to Congress spreadsheets. When new data are released, update that Jupyter notebook first. Data on rates were compiled in a separate spreadsheet.

In [267]:
phSt = pd.read_excel('pitHicAllStClean.xlsx', index_col=0); phSt.head()

,year,numCoCs,totalH,shelH,unshelH,totalIndiv,shelIndiv,unshelIndiv,totalInFam,shelInFam,...,rrhBedsDedVet,pshBedsDedVet,ophBedsDedVet,esBedsDedYouth,thBedsDedYouth,shBedsDedYouth,rrhBedsDedYouth,pshBedsDedYouth,ophBedsDedYouth,pshBedsDedCh
st,,,,,,,,,,,,,,,,,,,,,
AK,2007,2,1642,1387,255,1062,891,171,580,496,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149.0
AL,2007,8,5452,3796,1656,4184,2823,1361,1268,973,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,513.0
AR,2007,9,3836,2285,1551,2987,1766,1221,849,519,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0
AZ,2007,3,14646,8618,6028,10020,4423,5597,4626,4195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,865.0
CA,2007,42,138986,48511,90475,110952,30497,80455,28034,18014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7616.0


REMOVE STATES NOT GETTING A REPORT PAGE

In [268]:
phSt.drop(['GU','VI','PR','MP'], axis=0, inplace=True)

ADD NATIONAL DATA

In [269]:
phNtnl = pd.read_excel('pitHicNtnl.xlsx', index_col=0)
phNtnl['st'] = 'Ntnl'
phNtnl.reset_index(inplace=True)
phNtnl.set_index('st', inplace=True)
phNtnl.drop('numCoCs', axis=1, inplace=True)
phNtnl.head()

,year,totalH,shelH,unshelH,totalIndiv,shelIndiv,unshelIndiv,totalInFam,shelInFam,unshelInFam,...,rrhBedsDedVet,pshBedsDedVet,ophBedsDedVet,esBedsDedYouth,thBedsDedYouth,shBedsDedYouth,rrhBedsDedYouth,pshBedsDedYouth,ophBedsDedYouth,pshBedsDedCh
st,,,,,,,,,,,,,,,,,,,,,
Ntnl,2007,647258,391401,255857,412700,213073,199627,234558,178328,56230,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37793.0
Ntnl,2008,639784,386361,253423,404525,204855,199670,235259,181506,53753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42298.0
Ntnl,2009,630227,403308,226919,392131,215995,176136,238096,187313,50783,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50602.0
Ntnl,2010,637077,403543,233534,395140,212218,182922,241937,191325,50612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55256.0
Ntnl,2011,623788,392316,231472,387613,205834,181779,236175,186482,49693,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67964.0


ADD RATES PER CAPITA DATA

In [270]:
ratesSt = pd.read_excel('Rates by State.xlsx', index_col=0); ratesSt.head()
#These rates are with 2017 homelessness counts and 2016 Census population counts

,Rate,2016 population,Total Homeless,Sheltered,Unsheltered
ST,,,,,
AK,24.939341,739795.0,1845,1551,294
AL,7.780917,4874747.0,3793,2656,1137
AR,8.211621,3004279.0,2467,1273,1194
AZ,12.751790,7016270.0,8947,5781,3166
CA,33.962915,39536653.0,134278,42636,91642


**MERGE STATE AND NATIONAL DATA**

In [271]:
dfStNtnl = phSt.append(phNtnl)
dfStNtnl.tail()

,allBeds,esBedsAll,esBedsDedVet,esBedsDedYouth,esBedsNoChildren,esBedsNonDv,esBedsOnlyChildren,esBedsOverflow,esBedsSeasonal,esBedsWithChildren,...,unshelNonChronic,unshelNonVet,unshelParYouthAll,unshelParYouthChild,unshelParYouthYA,unshelVet,unshelYouthAll,unshelYouthChild,unshelYouthYA,year
st,,,,,,,,,,,,,,,,,,,,,
Ntnl,730376,238708,NaN,NaN,117885,203944,2716.0,27233,20822,118107,...,130406,0,NaN,NaN,NaN,20710,NaN,NaN,NaN,2013
Ntnl,762945,249497,NaN,NaN,123173,215115,3072.0,28331,22399,123252,...,116832,0,NaN,NaN,NaN,17570,NaN,NaN,NaN,2014
Ntnl,805252,264440,NaN,NaN,128575,230170,2858.0,22376,20791,133007,...,113760,0,439.0,8.0,431.0,16220,17069.0,2380.0,14689.0,2015
Ntnl,829332,264629,NaN,NaN,128315,230385,2791.0,17446,21368,133523,...,120333,0,520.0,2.0,518.0,13067,16498.0,1606.0,14892.0,2016
Ntnl,846957,277537,4756.0,6140.0,133344,242143,2843.0,15591,19833,141350,...,130065,0,577.0,13.0,564.0,15366,22257.0,2667.0,19590.0,2017


**PIVOT SUBSETS BY YEAR FOR SELECT OVER-TIME CALCS**

In [272]:
byYrTotalH = dfStNtnl.pivot(columns='year', values='totalH')
byYrTotalIndiv = dfStNtnl.pivot(columns='year', values='totalIndiv')
byYrTotalInFam = dfStNtnl.pivot(columns='year', values='totalInFam')
byYrTotalYouth = dfStNtnl.pivot(columns='year', values='totalYouthAll')
byYrTotalVet = dfStNtnl.pivot(columns='year', values='totalVet')
byYrTotalChronic = dfStNtnl.pivot(columns='year', values='totalChronic'); byYrTotalChronic.head()

year,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
st,,,,,,,,,,,
AK,278,439,323,164,253,279,184,206,222,122,257
AL,993,1052,1189,1043,1114,969,858,729,607,420,363
AR,852,893,406,395,607,437,602,519,623,503,473
AZ,2804,3097,2229,1935,2283,1895,1188,959,1366,1395,1552
CA,40341,40254,31412,31235,34589,33356,33112,31078,32227,31548,37360


CALCULATE CHANGES OVER TIME FROM PRIOR YEAR AND FROM 2010

In [273]:
latestYr = max(byYrTotalH.columns.values)
priorYr = int(latestYr) - 1

In [274]:
pops = [byYrTotalH, byYrTotalIndiv, byYrTotalInFam, byYrTotalYouth, byYrTotalVet, byYrTotalChronic]
popsNames = ["totalH", "totalIndiv", "totalInFam", "totalYouth", "totalVet", "totalChronic"]

In [275]:
i = 0
for p in pops:
    latestYr = max(p.columns.values)
    p['pctChgFromLastYr'] = round(((p.loc[:, latestYr] - p.loc[:, latestYr - 1]) / p.loc[:, latestYr - 1])*100, 1)
    p['pctChgFrom2010'] = round(((p.loc[:, latestYr] - p.loc[:, 2010]) / p.loc[:, 2010])*100, 1)
    p['numChgFromLastYr'] = p.loc[:, latestYr] - p.loc[:, latestYr - 1]
    p['numChgFrom2010'] = p.loc[:, latestYr] - p.loc[:, 2010]
    p.rename(columns={'pctChgFromLastYr': popsNames[i] + '_' + 'pctChgFromLastYr'}, inplace=True)
    p.rename(columns={'pctChgFrom2010': popsNames[i] + '_' + 'pctChgFrom2010'}, inplace=True)
    p.rename(columns={'numChgFromLastYr': popsNames[i] + '_' + 'numChgFromLastYr'}, inplace=True)
    p.rename(columns={'numChgFrom2010': popsNames[i] + '_' + 'numChgFrom2010'}, inplace=True)
    i += 1

In [276]:
byYrTotalH.head()

year,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,totalH_pctChgFromLastYr,totalH_pctChgFrom2010,totalH_numChgFromLastYr,totalH_numChgFrom2010
st,,,,,,,,,,,,,,,
AK,1642,1646,1992,1863,2128,1913,1946,1784,1956,1940,1845,-4.9,-1.0,-95,-18
AL,5452,5387,6080,6046,5558,5209,4689,4561,3970,4111,3793,-7.7,-37.3,-318,-2253
AR,3836,3255,2852,2762,3424,4214,3812,2936,2560,2463,2467,0.2,-10.7,4,-295
AZ,14646,12488,14721,13711,10504,11302,10562,10495,9896,9707,8947,-7.8,-34.7,-760,-4764
CA,138986,136531,123678,123480,125128,120098,118552,113952,115738,118142,134278,13.7,8.7,16136,10798


<a id='numeric'></a>
## Calculate the Numeric Variables

**BASIC TOTALS**

In [277]:
catsToInclude = ['totalH','unshelH','totalIndiv','totalInFam','totalYouthAll','totalVet','totalChronic']
mailMerge = dfStNtnl.loc[dfStNtnl['year'] == latestYr, catsToInclude]

In [278]:
mailMerge['totalYouthAll'] = mailMerge['totalYouthAll'].astype('int64')

**2016 COUNTS FOR STACKED BAR**

In [279]:
mailMerge['totalIndiv_priorYr'] = dfStNtnl.loc[dfStNtnl['year'] == priorYr, 'totalIndiv']
mailMerge['totalInFam_priorYr'] = dfStNtnl.loc[dfStNtnl['year'] == priorYr, 'totalInFam']
mailMerge['totalYouthAll_priorYr'] = dfStNtnl.loc[dfStNtnl['year'] == priorYr, 'totalYouthAll']
mailMerge['totalVet_priorYr'] = dfStNtnl.loc[dfStNtnl['year'] == priorYr, 'totalVet']
mailMerge['totalChronic_priorYr'] = dfStNtnl.loc[dfStNtnl['year'] == priorYr, 'totalChronic']

**PERCENT INCREASES OR DECREASES**

In [280]:
mailMerge = mailMerge.merge(byYrTotalH.loc[:,['totalH_pctChgFromLastYr','totalH_pctChgFrom2010','totalH_numChgFromLastYr','totalH_numChgFrom2010']], left_index=True, right_index=True)
mailMerge = mailMerge.merge(byYrTotalIndiv.loc[:,['totalIndiv_pctChgFromLastYr','totalIndiv_pctChgFrom2010','totalIndiv_numChgFromLastYr','totalIndiv_numChgFrom2010']], left_index=True, right_index=True)
mailMerge = mailMerge.merge(byYrTotalInFam.loc[:,['totalInFam_pctChgFromLastYr','totalInFam_pctChgFrom2010','totalInFam_numChgFromLastYr','totalInFam_numChgFrom2010']], left_index=True, right_index=True)
mailMerge = mailMerge.merge(byYrTotalYouth.loc[:,['totalYouth_pctChgFromLastYr','totalYouth_numChgFromLastYr']], left_index=True, right_index=True)
mailMerge = mailMerge.merge(byYrTotalVet.loc[:,['totalVet_pctChgFromLastYr','totalVet_pctChgFrom2010','totalVet_numChgFromLastYr','totalVet_numChgFrom2010']], left_index=True, right_index=True)
mailMerge = mailMerge.merge(byYrTotalChronic.loc[:,['totalChronic_pctChgFromLastYr','totalChronic_pctChgFrom2010','totalChronic_numChgFromLastYr','totalChronic_numChgFrom2010']], left_index=True, right_index=True)

**PERCENT UNSHELTERED**

In [281]:
mailMerge['pctUnshel'] = round((mailMerge['unshelH'] / mailMerge['totalH']) * 100,0)
mailMerge['pctShel'] = round((1 - (mailMerge['unshelH'] / mailMerge['totalH'])) * 100,0)

In [282]:
mailMerge['pctUnshel'] = mailMerge['pctUnshel'].astype('int')
mailMerge['pctShel'] = mailMerge['pctShel'].astype('int')

**RATES PER CAPITA**

In [283]:
mailMerge['rate'] = round(ratesSt['Rate'],0)

In [284]:
# Calculated for State of Homelessness
mailMerge.loc['Ntnl','rate'] = 17.0

In [285]:
mailMerge['rate'] = mailMerge['rate'].astype('int64')

**RANKINGS**

How does each state rank compared to the rest on change in the number of various subpopulation? These rankings will be used for for deciding what to include in the "what's working" and "what can work better" section at the bottom.

In [286]:
mailMerge['rankLoHi_totalH_pctChgFromLastYr'] = mailMerge.drop('Ntnl', axis=0)['totalH_pctChgFromLastYr'].rank(ascending=True)
mailMerge['rankLoHi_totalH_pctChgFrom2010'] = mailMerge.drop('Ntnl', axis=0)['totalH_pctChgFrom2010'].rank(ascending=True)
mailMerge['rankLoHi_totalH_numChgFromLastYr'] = mailMerge.drop('Ntnl', axis=0)['totalH_numChgFromLastYr'].rank(ascending=True)
mailMerge['rankLoHi_totalH_numChgFrom2010'] = mailMerge.drop('Ntnl', axis=0)['totalH_numChgFrom2010'].rank(ascending=True)

mailMerge['rankLoHi_totalIndiv_pctChgFromLastYr'] = mailMerge.drop('Ntnl', axis=0)['totalIndiv_pctChgFromLastYr'].rank(ascending=True)
mailMerge['rankLoHi_totalIndiv_pctChgFrom2010'] = mailMerge.drop('Ntnl', axis=0)['totalIndiv_pctChgFrom2010'].rank(ascending=True)
mailMerge['rankLoHi_totalIndiv_numChgFromLastYr'] = mailMerge.drop('Ntnl', axis=0)['totalIndiv_numChgFromLastYr'].rank(ascending=True)
mailMerge['rankLoHi_totalIndiv_numChgFrom2010'] = mailMerge.drop('Ntnl', axis=0)['totalIndiv_numChgFrom2010'].rank(ascending=True)

mailMerge['rankLoHi_totalInFam_pctChgFromLastYr'] = mailMerge.drop('Ntnl', axis=0)['totalInFam_pctChgFromLastYr'].rank(ascending=True)
mailMerge['rankLoHi_totalInFam_pctChgFrom2010'] = mailMerge.drop('Ntnl', axis=0)['totalInFam_pctChgFrom2010'].rank(ascending=True)
mailMerge['rankLoHi_totalInFam_numChgFromLastYr'] = mailMerge.drop('Ntnl', axis=0)['totalInFam_numChgFromLastYr'].rank(ascending=True)
mailMerge['rankLoHi_totalInFam_numChgFrom2010'] = mailMerge.drop('Ntnl', axis=0)['totalInFam_numChgFrom2010'].rank(ascending=True)

mailMerge['rankLoHi_totalYouth_pctChgFromLastYr'] = mailMerge.drop('Ntnl', axis=0)['totalYouth_pctChgFromLastYr'].rank(ascending=True)
mailMerge['rankLoHi_totalYouth_numChgFromLastYr'] = mailMerge.drop('Ntnl', axis=0)['totalYouth_numChgFromLastYr'].rank(ascending=True)

mailMerge['rankLoHi_totalVet_pctChgFromLastYr'] = mailMerge.drop('Ntnl', axis=0)['totalVet_pctChgFromLastYr'].rank(ascending=True)
mailMerge['rankLoHi_totalVet_pctChgFrom2010'] = mailMerge.drop('Ntnl', axis=0)['totalVet_pctChgFrom2010'].rank(ascending=True)
mailMerge['rankLoHi_totalVet_numChgFromLastYr'] = mailMerge.drop('Ntnl', axis=0)['totalVet_numChgFromLastYr'].rank(ascending=True)
mailMerge['rankLoHi_totalVet_numChgFrom2010'] = mailMerge.drop('Ntnl', axis=0)['totalVet_numChgFrom2010'].rank(ascending=True)

#not going to include changes from 2010 due to 2010 being only CH individuals
mailMerge['rankLoHi_totalChronic_pctChgFromLastYr'] = mailMerge.drop('Ntnl', axis=0)['totalChronic_pctChgFromLastYr'].rank(ascending=True)
#mailMerge['rankLoHi_totalChronic_pctChgFrom2010'] = mailMerge.drop('Ntnl', axis=0)['totalChronic_pctChgFrom2010'].rank(ascending=True)
mailMerge['rankLoHi_totalChronic_numChgFromLastYr'] = mailMerge.drop('Ntnl', axis=0)['totalChronic_numChgFromLastYr'].rank(ascending=True)
#mailMerge['rankLoHi_totalChronic_numChgFrom2010'] = mailMerge.drop('Ntnl', axis=0)['totalChronic_numChgFrom2010'].rank(ascending=True)

In [287]:
mailMerge.loc[:,['rankLoHi_totalInFam_pctChgFrom2010','totalInFam_pctChgFrom2010']].sort_values(by='rankLoHi_totalInFam_pctChgFrom2010').head(10)
#Note that if rank 13 and 14 are a tie, both would be labeled as 13.5!

,rankLoHi_totalInFam_pctChgFrom2010,totalInFam_pctChgFrom2010
st,,
LA,1.0,-68.7
NV,2.0,-66.3
WV,3.0,-62.4
KY,4.0,-60.2
OR,5.0,-59.8
CO,6.0,-57.4
NJ,7.0,-57.0
FL,8.0,-56.8
MD,9.0,-55.2


<a id='text'></a>
## Add the Text Variables

**STATE NAME**

In [288]:
states = {
        'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AZ': 'Arizona', 'CA': 'California',
        'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware',
        'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'IA': 'Iowa', 'ID': 'Idaho',
        'IL': 'Illinois', 'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana',
        'MA': 'Massachusetts', 'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota',
        'MO': 'Missouri', 'MS': 'Mississippi', 'MT': 'Montana', 'Ntnl':'America', 'NC': 'North Carolina',
        'ND': 'North Dakota', 'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
        'NM': 'New Mexico', 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio', 'OK': 'Oklahoma',
        'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
        'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VA': 'Virginia',
        'VT': 'Vermont', 'WA': 'Washington', 'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming'
}
mailMerge['state'] = pd.Series(states)

**STATE DEMONYM**

In [289]:
demonyms = {
        'AK': 'Alaskans', 'AL': 'Alabamians', 'AR': 'Arkansans', 'AZ': 'Arizonans', 'CA': 'Californians',
        'CO': 'Coloradans', 'CT': 'Connecticutters', 'DC': 'Washingtonians', 'DE': 'Delawareans',
        'FL': 'Floridians', 'GA': 'Georgians', 'HI': 'Islanders', 'IA': 'Iowans', 'ID': 'Idahoans',
        'IL': 'Illinoisans', 'IN': 'Hoosiers', 'KS': 'Kansans', 'KY': 'Kentuckians', 'LA': 'Louisianians',
        'MA': 'Massachusettsans', 'MD': 'Marylanders', 'ME': 'Mainers', 'MI': 'Michiganders', 'MN': 'Minnesotans',
        'MO': 'Missourians', 'MS': 'Mississippians', 'MT': 'Montanans', 'Ntnl':'Americans', 'NC': 'North Carolinians',
        'ND': 'North Dakotans', 'NE': 'Nebraskans', 'NH': 'New Hampshirites', 'NJ': 'New Jerseyans',
        'NM': 'New Mexicans', 'NV': 'Nevadans', 'NY': 'New Yorkers', 'OH': 'Ohioans', 'OK': 'Oklahomans',
        'OR': 'Oregonians', 'PA': 'Pennsylvanians', 'RI': 'Rhode Islanders', 'SC': 'South Carolinians',
        'SD': 'South Dakotans', 'TN': 'Tennesseans', 'TX': 'Texans', 'UT': 'Utahns', 'VA': 'Virginians',
        'VT': 'Vermonters', 'WA': 'Washingtonians', 'WI': 'Wisconsinites', 'WV': 'West Virginians', 'WY': 'Wyomingites'
}
mailMerge['demonym'] = pd.Series(demonyms)

**STATE RATE IS HOW MUCH HIGHER/LOWER THAN NATIONAL**

In [290]:
# National rate is...
ntnlRate = mailMerge.loc['Ntnl','rate']; ntnlRate
ntnlRateUnshel = mailMerge.loc['Ntnl','pctUnshel']

In [291]:
mailMerge['ntnlRate'] = ntnlRate.astype('int')
mailMerge['ntnlRateUnshel'] = ntnlRateUnshel.astype('int')

In [292]:
# Similar to = within 10% of national rate
# Lower than or higher = 10-50% lower or higher than national rate
# Significantly lower or higher = more than 50% lower or higher than national rate

def rateCompThresholds(val):
    if val < ntnlRate * 0.5:
        return 'significantly lower than'
    elif val < ntnlRate * 0.9:
        return 'lower than'
    elif val < ntnlRate * 1.1:
        return 'similar to'
    elif val < ntnlRate * 1.5:
        return 'higher than'
    else:
        return 'significantly higher than'

In [293]:
mailMerge['rateComp'] = mailMerge.drop('Ntnl', axis=0).apply(lambda row: rateCompThresholds(row['rate']), axis=1)
mailMerge.loc[:,['rate','rateComp']].head()

,rate,rateComp
st,,
AK,25,higher than
AL,8,significantly lower than
AR,8,significantly lower than
AZ,13,lower than
CA,34,significantly higher than


**PROGRESS SINCE 2010 AND 2016**

In [294]:
# Made significant progress = at least 20% reduction
# Made some progress = 1-20% reduction
# Not seen progress = 0% change or increase

def progressThresholds(val):
    if val <= -20:
        return 'made significant progress'
    elif val <= -1:
        return 'made some progress'
    else:
        return 'not seen progress'

In [295]:
mailMerge['progressFrom2010'] = mailMerge.apply(lambda row: progressThresholds(row['totalH_pctChgFrom2010']), axis=1)
mailMerge['progressFromLastYr'] = mailMerge.apply(lambda row: progressThresholds(row['totalH_pctChgFromLastYr']), axis=1)

**INCREASED/DECREASED**

In [296]:
def incrDecr(val):
    if val < 0:
        return 'decreased'
    elif val > 0:
        return 'increased'
    else:
        return 'changed'

In [297]:
mailMerge['totalH_incrDecrFrom2010'] = mailMerge.apply(lambda row: incrDecr(row['totalH_pctChgFrom2010']), axis=1)
mailMerge['totalH_incrDecrFromLastYr'] = mailMerge.apply(lambda row: incrDecr(row['totalH_pctChgFromLastYr']), axis=1)

In [298]:
def incrDecr2(val):
    if val < 0:
        return 'decreased'
    elif val > 0:
        return 'increased'

In [299]:
toIncrDecr = ['totalIndiv_pctChgFromLastYr', 'totalInFam_pctChgFromLastYr', 
           'totalYouth_pctChgFromLastYr', 'totalVet_pctChgFromLastYr', 'totalChronic_pctChgFromLastYr']
for i in toIncrDecr:
    newName = i.split(sep='_')[0] + '_incrDecrFromLastYr'
    mailMerge[newName] = mailMerge.apply(lambda row: incrDecr2(row[i]), axis=1)

**MOST PEOPLE ARE IN X SUBPOPULATION**

In [300]:
#Dictionary for column label to text name
columnToText = {'totalIndiv':'individual adults', 'totalInFam':'people in families with children',
               'totalVet':'veterans','totalYouthAll':'unaccompanied youth','totalChronic':'people experiencing chronic homelessness'}

In [301]:
mailMerge['mostPeopleAre'] = mailMerge.loc[:,['totalIndiv','totalInFam','totalYouthAll','totalVet','totalChronic']].idxmax(axis=1)
mailMerge['mostPeopleAre'].replace(to_replace=columnToText, inplace=True)

**SECOND MOST NUMBER OF PEOPLE ARE IN Y SUBPOPULATION**

In [302]:
def secondMost(a, b, c, d, e):
    vals = {a: 'individual adults', b:'people in families with children', c:'unaccompanied youth', d:'veterans', e:'people experiencing chronic homelessness'}
    return vals[sorted([a, b, c, d, e], reverse=True)[1]]

In [303]:
mailMerge['secondMostPeopleAre'] = mailMerge.apply(
    lambda row: secondMost(row['totalIndiv'], row['totalInFam'], row['totalYouthAll'], row['totalVet'], row['totalChronic']), axis=1)

** BEST AND WORST RANKING **

In [304]:
#Get subset of columns related to rankings
rank_cols = [col for col in mailMerge.columns if ('rankLoHi' in col and 'pctChg' in col)]

In [305]:
rankDescriptions = {'rankLoHi_totalH_pctChgFromLastYr':'Ranking - percent change in total homeless from prior year',
                       'rankLoHi_totalH_pctChgFrom2010':'Ranking - percent change in total homeless from 2010',
                       'rankLoHi_totalIndiv_pctChgFromLastYr':'Ranking - percent change in individuals from prior year',
                       'rankLoHi_totalIndiv_pctChgFrom2010':'Ranking - percent change in individuals from 2010',
                       'rankLoHi_totalInFam_pctChgFromLastYr':'Ranking - percent change in people in families from prior year',
                       'rankLoHi_totalInFam_pctChgFrom2010':'Ranking - percent change in people in families from 2010',
                       'rankLoHi_totalYouth_pctChgFromLastYr':'Ranking - percent change in youth from prior year',
                       'rankLoHi_totalVet_pctChgFromLastYr':'Ranking - percent change in veterans from prior year',
                       'rankLoHi_totalVet_pctChgFrom2010':'Ranking - percent change in veterans from 2010',
                       'rankLoHi_totalChronic_pctChgFromLastYr':'Ranking - percent change in chronically homeless from prior year',
                       'rankLoHi_totalH_numChgFromLastYr':'Ranking - number change in total homeless from prior year',
                       'rankLoHi_totalH_numChgFrom2010':'Ranking - number change in total homeless from 2010',
                       'rankLoHi_totalIndiv_numChgFromLastYr':'Ranking - number change in individuals from prior year',
                       'rankLoHi_totalIndiv_numChgFrom2010':'Ranking - number change in individuals from 2010',
                       'rankLoHi_totalInFam_numChgFromLastYr':'Ranking - number change in people in families from prior year',
                       'rankLoHi_totalInFam_numChgFrom2010':'Ranking - number change in people in families from 2010',
                       'rankLoHi_totalYouth_numChgFromLastYr':'Ranking - number change in youth from prior year',
                       'rankLoHi_totalVet_numChgFromLastYr':'Ranking - number change in veterans from prior year',
                       'rankLoHi_totalVet_numChgFrom2010':'Ranking - number change in veterans from 2010',
                       'rankLoHi_totalChronic_numChgFromLastYr':'Ranking - number change in chronically homeless from prior year'}

GET THE COLUMN FOR THE HIGHEST AND LOWEST RANKINGS

In [306]:
mailMerge['worstRanking'] = mailMerge.loc[:,rank_cols].idxmax(axis=1)
mailMerge['bestRanking'] = mailMerge.loc[:,rank_cols].idxmin(axis=1)
#Note that idxmax and idxmin return the first max or min, in case of multiple with the same ranking

CREATE THE COLUMNS NEEDED FOR CATEGORY, RANK, AND VALUE

In [307]:
states = list(mailMerge.index.values)
states.remove('Ntnl') #has Nan rankings

for st in states:
    #Best rankings
    bestRank = mailMerge.loc[st,'bestRanking']
    bestRankLabel = bestRank[9:]
    mailMerge.loc[st,'bestRankingRank'] = mailMerge.loc[st,bestRank]
    mailMerge.loc[st,'bestRankingValue'] = mailMerge.loc[st,bestRankLabel]
    #Worst rankings
    worstRank = mailMerge.loc[st,'worstRanking']
    worstRankLabel = worstRank[9:]
    mailMerge.loc[st,'worstRankingRank'] = mailMerge.loc[st,worstRank]
    mailMerge.loc[st,'worstRankingValue'] = mailMerge.loc[st,worstRankLabel]

CONVERT TO A DESCRIPTIVE TEXT

In [308]:
mailMerge['bestRanking'] = mailMerge['bestRanking'].map(lambda x: rankDescriptions[str(x)], na_action='ignore')
mailMerge['worstRanking'] = mailMerge['worstRanking'].map(lambda x: rankDescriptions[str(x)], na_action='ignore')

INDICATE WHICH YEAR THE HIGH/LOW RANKED PROGRESS IS FOR

In [309]:
mailMerge['needsImprovementYear'] = np.where(mailMerge['worstRanking'].map(lambda x: '2010' in str(x), na_action='ignore'), '2010', '2016')
mailMerge['makingProgressYear'] = np.where(mailMerge['bestRanking'].map(lambda x: '2010' in str(x), na_action='ignore'), '2010', '2016')
mailMerge.loc['Ntnl',['needsImprovementYear','makingProgressYear']] = np.nan

GET THE SUBPOPULATION CATEGORY AND RENAME TO READ WELL

In [310]:
mailMerge['needsImprovementCategory'] = mailMerge['worstRanking'].map(lambda x: (re.search(r'change in(.*?) from ', str(x)).group(1)).strip(), na_action='ignore')
mailMerge['makingProgressCategory'] = mailMerge['bestRanking'].map(lambda x: (re.search(r'change in(.*?) from ', str(x)).group(1)).strip(), na_action='ignore')

In [311]:
renameForCategory = {'individuals':'for individual adults',
                    'people in families':'for people in families',
                    'veterans':'for veterans',
                    'total homeless':'overall',
                    'chronically homeless':'for people experiencing chronic homelessness',
                    'youth':'for youth'}

In [312]:
mailMerge['needsImprovementCategory'] = mailMerge['needsImprovementCategory'].map(lambda x: renameForCategory[x], na_action='ignore')
mailMerge['makingProgressCategory'] = mailMerge['makingProgressCategory'].map(lambda x: renameForCategory[x], na_action='ignore')

In [313]:
mailMerge.head(3)

,totalH,unshelH,totalIndiv,totalInFam,totalYouthAll,totalVet,totalChronic,totalIndiv_priorYr,totalInFam_priorYr,totalYouthAll_priorYr,...,worstRanking,bestRanking,bestRankingRank,bestRankingValue,worstRankingRank,worstRankingValue,needsImprovementYear,makingProgressYear,needsImprovementCategory,makingProgressCategory
st,,,,,,,,,,,,,,,,,,,,,
AK,1845,294,1354,491,162,124,257,1527,413,173.0,...,Ranking - percent change in chronically homele...,Ranking - percent change in individuals from p...,7.0,-11.3,50.0,110.7,2016,2016,for people experiencing chronic homelessness,for individual adults
AL,3793,1137,2985,808,294,269,363,3019,1092,254.0,...,Ranking - percent change in youth from prior year,Ranking - percent change in people in families...,4.0,-26.0,40.0,15.7,2016,2016,for youth,for people in families
AR,2467,1194,2068,399,208,239,473,1838,625,201.0,...,Ranking - percent change in individuals from p...,Ranking - percent change in people in families...,2.0,-36.2,44.0,12.5,2016,2016,for individual adults,for people in families


<a id='chartwell'></a>
## Add Columns for InDesign Graphs
### and Format Numbers as Text if Needed

** MAKING BAR GRAPH COUNTS PROPORTIONAL TO THE INDESIGN RANGE (100 IN THIS CASE)**

In [314]:
#Get the maximum of current or prior year across all subpopulations
#This maximum will be set equal to 100 and everything else will be proportional to it
mailMerge['maxVal'] = mailMerge[['totalIndiv', 'totalInFam', 'totalYouthAll', 'totalVet', 'totalChronic',
    'totalIndiv_priorYr', 'totalInFam_priorYr', 'totalYouthAll_priorYr', 'totalVet_priorYr', 
    'totalChronic_priorYr']].max(axis=1)

In [315]:
mailMerge['barLatestTotalIndiv'] = round((mailMerge['totalIndiv'] / mailMerge['maxVal']) * 100, 0).astype('int')
mailMerge['barLatestTotalInFam'] = round((mailMerge['totalInFam'] / mailMerge['maxVal']) * 100, 0).astype('int')
mailMerge['barLatestTotalYouth'] = round((mailMerge['totalYouthAll'] / mailMerge['maxVal']) * 100, 0).astype('int')
mailMerge['barLatestTotalVet'] = round((mailMerge['totalVet'] / mailMerge['maxVal']) * 100, 0).astype('int')
mailMerge['barLatestTotalChronic'] = round((mailMerge['totalChronic'] / mailMerge['maxVal']) * 100, 0).astype('int')
mailMerge['barPriorTotalIndiv'] = round((mailMerge['totalIndiv_priorYr'] / mailMerge['maxVal']) * 100, 0).astype('int')
mailMerge['barPriorTotalInFam'] = round((mailMerge['totalInFam_priorYr'] / mailMerge['maxVal']) * 100, 0).astype('int')
mailMerge['barPriorTotalYouth'] = round((mailMerge['totalYouthAll_priorYr'] / mailMerge['maxVal']) * 100, 0).astype('int')
mailMerge['barPriorTotalVet'] = round((mailMerge['totalVet_priorYr'] / mailMerge['maxVal']) * 100, 0).astype('int')
mailMerge['barPriorTotalChronic'] = round((mailMerge['totalChronic_priorYr'] / mailMerge['maxVal']) * 100, 0).astype('int')

SETTING THEM IN THE FORMAT NEEDED (##+##+##+##)

In [316]:
mailMerge['stackedBarsPrior'] = mailMerge['barPriorTotalIndiv'].map(str)+'+'+mailMerge['barPriorTotalInFam'].map(str)+'+'+mailMerge['barPriorTotalYouth'].map(str)+'+'+mailMerge['barPriorTotalVet'].map(str)+'+'+mailMerge['barPriorTotalChronic'].map(str)
mailMerge['stackedBarsLatest'] = mailMerge['barLatestTotalIndiv'].map(str)+'+'+mailMerge['barLatestTotalInFam'].map(str)+'+'+mailMerge['barLatestTotalYouth'].map(str)+'+'+mailMerge['barLatestTotalVet'].map(str)+'+'+mailMerge['barLatestTotalChronic'].map(str)

** BASIC COUNTS AS STRINGS WITH COMMA SEPARATOR**

In [317]:
columnsToString = ['totalH', 'pctUnshel', 'pctShel', 'unshelH', 'totalIndiv', 'totalInFam', 
                   'totalYouthAll', 'totalVet', 'totalChronic']
for c in columnsToString:
    mailMerge[c] = mailMerge[c].apply(lambda row: '{:,}'.format(row))

**PERCENT CHANGES AS ABSOLUTE VALUES**

In [318]:
toAbsVal = ['totalH_pctChgFrom2010', 'totalH_pctChgFromLastYr', 'totalIndiv_pctChgFromLastYr', 'totalInFam_pctChgFromLastYr', 
           'totalYouth_pctChgFromLastYr', 'totalVet_pctChgFromLastYr', 'totalChronic_pctChgFromLastYr']
for i in toAbsVal:
    newName = 'abs_' + i
    mailMerge[newName] = abs(mailMerge[i])

<a id='simulation'></a>
## Run a Sample Text Simulation
Test several states with the static and variable numbers/text as if doing the mail merge into InDesign

In [324]:
def stateReportSimulation(stateAbbrevString):
    stateSim = stateAbbrevString
    state = mailMerge.loc[stateSim, 'state']
    print('How many people are homeless in', state, '\b?')
    print('There are', mailMerge.loc[stateSim, 'totalH'], 'people experiencing homelessness in', 
          state, 'on any given night. That\'s a rate of', mailMerge.loc[stateSim, 'rate'],
         'in every 10,000 people. Across the Unites States, 17 in every 10,000 people are homeless, putting',
          mailMerge.loc[stateSim, 'state'], '\b\'s rate', mailMerge.loc[stateSim, 'rateComp'], 
          'than the national average.')
    print('--------------------------')
    print('CHART:')
    print(mailMerge.loc[stateSim, 'totalH'], mailMerge.loc[stateSim, 'demonym'], 'experiencing homelessness')
    print('--------------------------')
    print('Is homelessness up or down in', state, '\b?')
    print(state, 'has', mailMerge.loc[stateSim, 'progressFrom2010'],
         'toward ending homelessness since 2010 -- overall homelessness has', 
         mailMerge.loc[stateSim, 'totalH_incrDecrFrom2010'], mailMerge.loc[stateSim, 'abs_totalH_pctChgFrom2010'],
         '\b%. Between 2016 and 2017, homelessness', mailMerge.loc[stateSim, 'totalH_incrDecrFromLastYr'],
         mailMerge.loc[stateSim, 'abs_totalH_pctChgFromLastYr'], '\b%.')
    print('--------------------------')
    print('Who experiences homelessness in', state, '\b?')
    print('Most people experiencing homenessness in',  state, 'are', mailMerge.loc[stateSim, 'mostPeopleAre'],
         '\b, followed by', mailMerge.loc[stateSim, 'secondMostPeopleAre'],
         '\b.')
    print('--------------------------')
    print(mailMerge.loc[stateSim, 'totalIndiv'], 'individual adults |', 
          mailMerge.loc[stateSim, 'totalIndiv_incrDecrFromLastYr'], 
          mailMerge.loc[stateSim, 'abs_totalIndiv_pctChgFromLastYr'], '\b% since 2016')
    print(mailMerge.loc[stateSim, 'totalInFam'], 'people in families |', 
          mailMerge.loc[stateSim, 'totalInFam_incrDecrFromLastYr'], 
          mailMerge.loc[stateSim, 'abs_totalInFam_pctChgFromLastYr'], '\b%')
    print(mailMerge.loc[stateSim, 'totalYouthAll'], 'unaccompanied youth |', 
          mailMerge.loc[stateSim, 'totalYouth_incrDecrFromLastYr'], 
          mailMerge.loc[stateSim, 'abs_totalYouth_pctChgFromLastYr'], '\b%')
    print(mailMerge.loc[stateSim, 'totalVet'], 'veterans |', 
          mailMerge.loc[stateSim, 'totalVet_incrDecrFromLastYr'], 
          mailMerge.loc[stateSim, 'abs_totalVet_pctChgFromLastYr'], '\b%')
    print(mailMerge.loc[stateSim, 'totalChronic'], 'chronically homeless |', 
          mailMerge.loc[stateSim, 'totalChronic_incrDecrFromLastYr'], 
          mailMerge.loc[stateSim, 'abs_totalChronic_pctChgFromLastYr'], '\b%')
    print('--------------------------')
    print('CHART:')
    print(mailMerge.loc[stateSim, 'stackedBarsPrior'])
    print(mailMerge.loc[stateSim, 'stackedBarsLatest'])
    print('--------------------------')
    print('What does homelessness look like in', state, '\b?')
    print(mailMerge.loc[stateSim, 'pctUnshel'],'\b% of people experiencing homelessness in',
         state, 'live "unsheltered" in a place not meant for human habitation',
         '-- like a park, car, or abandoned building. That\'s lower than the national unsheltered rate of',
         mailMerge.loc[stateSim, 'ntnlRateUnshel'], '\b%.')
    print('--------------------------')
    print('CHART:')
    print(mailMerge.loc[stateSim, 'pctUnshel'], '\b% unsheltered in', state)
    print(mailMerge.loc[stateSim, 'pctUnshel'], '|', mailMerge.loc[stateSim, 'pctShel'])

In [325]:
stateReportSimulation('DC')

How many people are homeless in District of Columbia ?
There are 7,473 people experiencing homelessness in District of Columbia on any given night. That's a rate of 108 in every 10,000 people. Across the Unites States, 17 in every 10,000 people are homeless, putting District of Columbia 's rate significantly higher than than the national average.
--------------------------
CHART:
7,473 Washingtonians experiencing homelessness
--------------------------
Is homelessness up or down in District of Columbia ?
District of Columbia has not seen progress toward ending homelessness since 2010 -- overall homelessness has increased 14.3 %. Between 2016 and 2017, homelessness decreased 10.5 %.
--------------------------
Who experiences homelessness in District of Columbia ?
Most people experiencing homenessness in District of Columbia are people in families with children , followed by individual adults .
--------------------------
3,583 individual adults | decreased 2.7 % since 2016
3,890

In [326]:
stateReportSimulation('VA')

How many people are homeless in Virginia ?
There are 6,067 people experiencing homelessness in Virginia on any given night. That's a rate of 7 in every 10,000 people. Across the Unites States, 17 in every 10,000 people are homeless, putting Virginia 's rate significantly lower than than the national average.
--------------------------
CHART:
6,067 Virginians experiencing homelessness
--------------------------
Is homelessness up or down in Virginia ?
Virginia has made significant progress toward ending homelessness since 2010 -- overall homelessness has decreased 33.2 %. Between 2016 and 2017, homelessness decreased 3.2 %.
--------------------------
Who experiences homelessness in Virginia ?
Most people experiencing homenessness in Virginia are individual adults , followed by people in families with children .
--------------------------
4,028 individual adults | increased 1.7 % since 2016
2,039 people in families | decreased 11.7 %
303 unaccompanied youth | decreased 5.9 %
4

<a id='format'></a>
## Format the Data for Other People

** KEEP ONLY THE COLUMNS NEEDED TO RENDER THE REPORT  **

**AND... BE NICE AND PUT THEM IN THE ORDER THEY APPEAR IN THE REPORT  **

In [321]:
columnsNeeded = ['state','totalH','rate','ntnlRate','rateComp','demonym','progressFrom2010',
                'totalH_incrDecrFrom2010','abs_totalH_pctChgFrom2010','totalH_incrDecrFromLastYr',
                'abs_totalH_pctChgFromLastYr','mostPeopleAre','secondMostPeopleAre',
                'totalIndiv','totalIndiv_incrDecrFromLastYr','abs_totalIndiv_pctChgFromLastYr',
                'totalInFam','totalInFam_incrDecrFromLastYr','abs_totalInFam_pctChgFromLastYr',
                'totalYouthAll','totalYouth_incrDecrFromLastYr','abs_totalYouth_pctChgFromLastYr',
                'totalVet','totalVet_incrDecrFromLastYr','abs_totalVet_pctChgFromLastYr',
                'totalChronic','totalChronic_incrDecrFromLastYr','abs_totalChronic_pctChgFromLastYr',
                'barLatestTotalIndiv','barPriorTotalIndiv','barLatestTotalInFam','barPriorTotalInFam',
                'barLatestTotalYouth','barPriorTotalYouth','barLatestTotalVet','barPriorTotalVet',
                'barLatestTotalChronic','barPriorTotalChronic','pctUnshel','ntnlRateUnshel',
                'needsImprovementCategory', 'needsImprovementYear',
                'makingProgressCategory', 'makingProgressYear',
                'bestRanking', 'bestRankingRank', 'bestRankingValue', 'worstRanking', 'worstRankingRank', 
                'worstRankingValue']
mailMergeNeededOnly = mailMerge[columnsNeeded].copy()

** ADD MORE DESCRIPTIVE HEADERS **

In [322]:
renameColumns = {'state':'state',
                 'totalH':'count of total homeless in latest year',
                 'rate':'rate',
                 'ntnlRate':'national rate',
                 'rateComp':'text for comparison to national rate',
                 'demonym':'demonym',
                 'progressFrom2010':'text for significant progress',
                 'totalH_incrDecrFrom2010':'text for increase or decrease in total homeless from 2010',
                 'abs_totalH_pctChgFrom2010':'percent change (abs val) in total homeless from 2010',
                 'totalH_incrDecrFromLastYr':'text for increase or decrease in total homeless from prior year',
                 'abs_totalH_pctChgFromLastYr':'percent change (abs val) in total homeless from prior year',
                 'mostPeopleAre':'text for category most people are',
                 'secondMostPeopleAre':'text for category second most people are',
                 'totalIndiv':'count of individuals in latest year',
                 'totalIndiv_incrDecrFromLastYr':'text for increase or decrease in individuals from prior year',
                 'abs_totalIndiv_pctChgFromLastYr':'percent change (abs val) in individuals from prior year',
                 'totalInFam':'count of people in families in latest year',
                 'totalInFam_incrDecrFromLastYr':'text for increase or decrease in people in families from prior year',
                 'abs_totalInFam_pctChgFromLastYr':'percent change (abs val) in people in families from prior year',
                 'totalYouthAll':'count of youth in latest year',
                 'totalYouth_incrDecrFromLastYr':'text for increase or decrease in youth from prior year',
                 'abs_totalYouth_pctChgFromLastYr':'percent change (abs val) in youth from prior year',
                 'totalVet':'count of veterans in latest year',
                 'totalVet_incrDecrFromLastYr':'text for increase or decrease in veterans from prior year',
                 'abs_totalVet_pctChgFromLastYr':'percent change (abs val) in veterans from prior year',
                 'totalChronic':'count of chronically homeless in latest year',
                 'totalChronic_incrDecrFromLastYr':'text for increase or decrease in chronically homeless from prior year',
                 'abs_totalChronic_pctChgFromLastYr':'percent change (abs val) in chronically homeless from prior year',
                 'barLatestTotalIndiv':'bar chart value for individuals in latest year',
                 'barPriorTotalIndiv':'bar chart value for individuals in prior year',
                 'barLatestTotalInFam':'bar chart value for people in families in latest year',
                 'barPriorTotalInFam':'bar chart value for people in families in prior year',
                 'barLatestTotalYouth':'bar chart value for youth in latest year',
                 'barPriorTotalYouth':'bar chart value for youth in prior year',
                 'barLatestTotalVet':'bar chart value for veterans in latest year',
                 'barPriorTotalVet':'bar chart value for veterans in prior year',
                 'barLatestTotalChronic':'bar chart value for chronically homeless in latest year',
                 'barPriorTotalChronic':'bar chart value for chronically homeless in prior year',
                 'pctUnshel':'percent unsheltered',
                 'ntnlRateUnshel':'national percent unsheltered',
                 'needsImprovementCategory':'category for needs improvement', 
                 'needsImprovementYear': 'year for needs improvement',
                 'makingProgressCategory': 'category for making progress', 
                 'makingProgressYear': 'year for making progress',
                 'bestRanking':'Best ranking trend - category',
                 'bestRankingRank':'Best ranking trend - rank', 
                 'bestRankingValue':'Best ranking trend - value', 
                 'worstRanking':'Worst ranking trend - category', 
                 'worstRankingRank':'Worst ranking trend - rank',
                 'worstRankingValue':'Worst ranking trend - value'
}

In [323]:
mailMergeNeededOnly.rename(columns=renameColumns, inplace=True)
mailMergeNeededOnly.head(1)

,state,count of total homeless in latest year,rate,national rate,text for comparison to national rate,demonym,text for significant progress,text for increase or decrease in total homeless from 2010,percent change (abs val) in total homeless from 2010,text for increase or decrease in total homeless from prior year,...,category for needs improvement,year for needs improvement,category for making progress,year for making progress,Best ranking trend - category,Best ranking trend - rank,Best ranking trend - value,Worst ranking trend - category,Worst ranking trend - rank,Worst ranking trend - value
st,,,,,,,,,,,,,,,,,,,,,
AK,Alaska,"1,845",25,17,higher than,Alaskans,made some progress,decreased,1.0,decreased,...,for people experiencing chronic homelessness,2016,for individual adults,2016,Ranking - percent change in individuals from p...,7.0,-11.3,Ranking - percent change in chronically homele...,50.0,110.7


<a id='export'></a>
## Export
to Excel for the Mail Merge into InDesign